In [0]:
%run "../includes/paths"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("P_data_source","")
v_data_source = dbutils.widgets.get("P_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import *
circuits_schema = StructType([StructField("circuit_id", IntegerType(), False),
                            StructField("circuit_ref", StringType(), True), 
                            StructField("name", StringType(), True), 
                            StructField("location", StringType(), True), 
                            StructField("country", StringType(), True), 
                            StructField("lat", DoubleType(), True), 
                            StructField("lng", DoubleType(), True), 
                            StructField("alt", IntegerType(), True), 
                            StructField("url", StringType(), True)])

In [0]:
circuits_df = spark.read.option("header",True).schema(circuits_schema).csv(f"{raw_container_folder_path}/{v_file_date}/circuits.csv")

In [0]:
from pyspark.sql.functions import col, lit

circuits_selected_df = circuits_df.select(col("circuit_id"), col("circuit_ref"), col("name"), col("location"), col("country"), col("lat"), col("lng"), col("alt").alias("altitude"))

In [0]:
circuits_renamed_df = circuits_selected_df.withColumnRenamed("lat", "latitude").withColumnRenamed("lng", "longitude")

In [0]:
from pyspark.sql.functions import current_timestamp
circuits_final_df = circuits_renamed_df.withColumn("ingestion_date", current_timestamp())\
                                        .withColumn("data_source", lit(v_data_source))\
                                        .withColumn("file_date",lit(v_file_date))

In [0]:
circuits_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed_db.circuits")

In [0]:
%sql
select * from f1_processed_db.circuits

In [0]:
%sql
DESC HISTORY f1_processed_db.circuits